### Ingest constructors.json file

##### Read the json file using spark df reader

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType
from pyspark.sql.functions import col, current_timestamp, lit, to_timestamp,concat

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructor_df = spark.read \
.option("header", "true") \
.schema(constructors_schema) \
.json(f"{raw_folder_path}/constructors.json")

##### Select the columns

In [0]:
constructor_dropped_df = constructor_df.drop(col("url"))

In [0]:
constructor_final_df = constructor_dropped_df.withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("constructorRef", "constructor_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("data_source", lit(v_data_source))
                                          

Write the result as parquet file

In [0]:
#constructor_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")

In [0]:
constructor_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.constructors")

In [0]:
#%fs
#ls /mnt/formula1dlstudy/processed/constructors

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/constructors"))

In [0]:
%sql
SELECT * FROM f1_processed.constructors

In [0]:
dbutils.notebook.exit("Success")